### Task 1: Random Forest
#### **Objective**
In this lab, you will implement missing parts of a `RandomForest` classifier. This classifier is an ensemble of decision trees that are trained using bootstrap sampling and random subspace method for feature selection. (**Note:** This is not a *trditional* Random Forest, since we are using the random subspace method — each *tree* sees a subsample of features — instead of the random attribute method, where each *node* sees a subsample of features.)

#### **Overview**
A `RandomForest` model is built using the following steps:
1. **Bootstrap Sampling**: Randomly selecting samples with replacement to train each tree.
2. **Random Feature Selection**: Choosing a subset of features for training each tree.
3. **Training Decision Trees**: Fitting multiple decision trees using the sampled dataset.
4. **Aggregating Predictions**: Using majority voting to determine the final prediction.

Your task is to implement the missing parts (marked as `TODO`) in the given code.

---

#### **SubTasks**

##### **SubTask 1: Implement Bootstrap Sampling**
- **Function to complete**: `_bootstrap_sample(self, X, y)`
- **Purpose**: Generate a bootstrap sample by randomly selecting data points **with replacement**.
- **Implementation details**:
  - Randomly sample `len(X)` instances from `X` and `y`, allowing repetition.
  - Return:
    - `X_sample`: The sampled feature matrix.
    - `y_sample`: The corresponding labels.
    - `indices`: The indices of the selected samples.
- **Steps**:
  1. Use `np.random.choice()` to sample indices from `X`.
  2. Select the corresponding rows from `X` and `y` using the sampled indices.
  3. Return `(X_sample, y_sample, indices)`.

**Hints:**
- Use `replace=True` in `np.random.choice()` to allow repeated selections.
- The number of selected indices should be equal to the number of rows in `X`.

---

##### **SubTask 2: Implement Random Feature Selection**
- **Function to complete**: `_select_random_features(self, X, n_features)`
- **Purpose**: Select a subset of features randomly to be used for training each tree.
- **Implementation details**:
  - Determine the number of features to select based on `self.max_features`:
    - `"sqrt"` → `sqrt(n_features)`
    - `"log2"` → `log2(n_features)`
    - `int` value → Use the given number of features.
  - Randomly select `max_features` feature indices.
  - Return:
    - `X_subset`: The feature matrix containing only the selected features.
    - `feature_indices`: The indices of the selected features.
- **Steps**:
  1. Compute the number of features to select (`max_features`).
  2. Randomly choose `max_features` feature indices using `np.random.choice()`.
  3. Extract the corresponding columns from `X` using these indices.
  4. Return `(X_subset, feature_indices)`.

**Hints:**
- Ensure `max_features` does not exceed `n_features`.
- Use `replace=False` in `np.random.choice()` to prevent selecting the same feature multiple times.

---

##### **SubTask 3: Train Individual Decision Trees**
- **Function to complete**: `fit(self, X, y)`
- **Purpose**: Train `n_estimators` decision trees using bootstrap samples and random feature selection.
- **Implementation details**:
  - **For each tree:**
    1. **Create a bootstrap sample** from `X` and `y` using `_bootstrap_sample()`.
    2. **Select a subset of features** from the bootstrap sample using `_select_random_features()`.
    3. **Train a `DecisionTreeClassifier`** using the subset of features.
    4. Store the trained tree and selected feature indices.

**Steps:**
1. Convert `X` and `y` into NumPy arrays if they are Pandas DataFrames/Series.
2. Initialize an empty list `self.trees` to store trained trees.
3. For each tree (`self.n_estimators`):
   - Call `_bootstrap_sample()` to generate a training sample.
   - Call `_select_random_features()` to select features for training.
   - Train a `DecisionTreeClassifier` on the sampled data.
   - Store the trained tree and the selected feature indices in `self.trees`.

**Hints:**
- Use `np.random.seed(self.seed)` for reproducibility.

In [7]:
import numpy as np
from collections import Counter
from sklearn.tree import DecisionTreeClassifier


class RandomForest:
    """
    A basic implementation of a Random Forest algorithm for classification.

    This class builds an ensemble of decision trees using bootstrap sampling and random feature selection.
    It combines predictions from individual trees using majority voting to produce a robust classification model.

    Attributes:
    ----------
    n_estimators : int
        Number of decision trees in the forest.
    max_depth : int or None
        Maximum depth of each tree. If None, the trees grow until pure leaves or until reaching `min_samples_split`.
    min_samples_split : int
        Minimum number of samples required to split an internal node in a tree.
    max_features : str or int
        Number of features to consider when looking for the best split.
        - "sqrt": Square root of the total number of features.
        - "log2": Base-2 logarithm of the total number of features.
        - int: A specific number of features to use.
    seed : int or None
        Random seed for reproducibility. If None, results will vary between runs.
    trees : list
        A list containing tuples of trained decision trees and their selected feature indices/names.
    """

    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features="sqrt", seed=None):
        """
        Initializes the Random Forest classifier.

        Parameters:
        ----------
        n_estimators : int, default=10
            Number of decision trees to build in the forest.
        max_depth : int or None, default=None
            Maximum depth of each decision tree. If None, trees grow until they are pure or cannot be split further.
        min_samples_split : int, default=2
            Minimum number of samples required to split an internal node.
        max_features : str or int, default="sqrt"
            Number of features to consider when splitting nodes:
            - "sqrt": Square root of the total number of features.
            - "log2": Base-2 logarithm of the total number of features.
            - int: Specific number of features.
        seed : int or None, default=None
            Random seed for reproducibility. If None, results may vary across runs.
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.seed =  seed
        self.trees = []

    def _bootstrap_sample(self, X, y):
        """
        Generates a bootstrap sample of the input dataset by sampling with replacement.

        Parameters:
        ----------
        X : numpy.ndarray
            Feature matrix with shape (n_samples, n_features).
        y : numpy.ndarray
            Target vector with shape (n_samples,).

        Returns:
        -------
        tuple
            A tuple (X_sample, y_sample, indices) where:
            - X_sample is the feature matrix of the bootstrap sample.
            - y_sample is the target vector of the bootstrap sample.
            - indices are the indices of the selected samples.
        """
        # TODO implement this function
        # Step 1: Use np.random.choice() to sample indices from X.
        indices = np.random.choice(X.shape[0],
                                   X.shape[0],
                                   replace=True)

        # Step 2: Select the corresponding rows from X and y using the sampled indices.
        X_sample = X[indices]
        y_sample = y[indices]

        return X_sample, y_sample, indices

    def _select_random_features(self, X, n_features):
        """
        Selects a random subset of features to use for a decision tree.

        Parameters:
        ----------
        X : numpy.ndarray
            Feature matrix with shape (n_samples, n_features).
        n_features : int
            Total number of features in the dataset.

        Returns:
        -------
        tuple
            A tuple (X_subset, feature_indices) where:
            - X_subset is the feature matrix containing only the selected features.
            - feature_indices are the indices of the selected features.
        """
        # TODO implement this function
        # Step 1: Compute the number of features to select (max_features).
        if self.max_features == "sqrt":
            max_features = np.sqrt(n_features)
        elif self.max_features == "log2":
            max_features = np.log2(n_features)
        else:
            max_features = 5;

        # Step 2: Randomly choose max_features feature indices using np.random.choice().
        feature_indices = np.random.choice(n_features, size=int(max_features), replace=False)

        # Step 3: Extract the corresponding columns from X using these indices.
        X_subset = X[:,feature_indices]
        return X_subset, feature_indices

    def fit(self, X, y):
        """
        Trains the Random Forest model by creating an ensemble of decision trees.

        Each tree is trained on a bootstrap sample of the data and a random subset of features.

        Parameters:
        ----------
        X : numpy.ndarray
            Feature matrix with shape (n_samples, n_features).
        y : numpy.ndarray
            Target vector with shape (n_samples,).
        """
        self.feature_names = None

        self.trees = []
        n_features = X.shape[1]
        if self.seed is not None:
            np.random.seed(self.seed)


        for _ in range(self.n_estimators):
            # TODO implement this function
            # Create bootstrap sample
            # Step 1: Call _bootstrap_sample() to generate a training sample.
            X_sample, y_sample, _ = self._bootstrap_sample(X, y)
            # Step 2: Call _select_random_features() to select features for training.
            X_subset, feature_indices = self._select_random_features(X_sample,n_features)

            # Step 3: Train a DecisionTreeClassifier on the sampled data.
            tree = DecisionTreeClassifier(
                        max_depth=self.max_depth,
                        min_samples_split=self.min_samples_split,
                        random_state=self.seed
                        )
            tree.fit(X_subset, y_sample)

            # Step 4: Store the trained tree and the selected feature indices in self.trees.
            selected_features = feature_indices
            self.trees.append((tree, selected_features))

    def predict(self, X):
        """
        Predicts class labels for the input data using the trained Random Forest.

        Predictions are made by aggregating votes from all individual trees (majority voting).

        Parameters:
        ----------
        X : numpy.ndarray
            Feature matrix with shape (n_samples, n_features).

        Returns:
        -------
        numpy.ndarray
            Predicted class labels of shape (n_samples,).
        """

        predictions = np.zeros((X.shape[0], len(self.trees)))
        for i, (tree, selected_features) in enumerate(self.trees):
            feature_indices = selected_features

            predictions[:, i] = tree.predict(X[:, feature_indices])

        return np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=1, arr=predictions)

In [6]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, accuracy_score

# Step 1: Load Dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split the data into a training and a test set
X_mix, X_val, y_mix, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X_mix, y_mix, test_size=0.25, random_state=0, stratify=y_mix)
rf = RandomForest(n_estimators=5, max_depth=5, max_features="sqrt", seed=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.956140350877193